# Generalized Machine Learning 

- A spatial description of the ML problem 
- The bias/variance trade-off 
- Model selection and search 
- Generalized machine learning 
- Deploying machine learning models

In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

sns.set_context('notebook')
sns.set_style('whitegrid')

## The Machine Learning Problem

### Learning from Examples

### Terminology

### Categorized by Input Type

### Categorized by Output Type

## Regression

### Hypothesis

### Example

### Evaluation

## Classification

### Hypothesis

### Example

### Evaluation

## Clustering

### Hypothesis

### Example

### Evaluation

## Generalized Machine Learning

### Features

### Models

### Parameters and Hyperparameters

## Machine Learning API

### Estimators

### Models

### Transformers

### Pipelines

## Model Selection

### Model Selection Triple

### A Search Problem

### Visual Steering

## Complexity

### Underfitting

### Overfitting

### Cross-Validation

### Bias and Variance

### Sources of Complexity

### Interpretability

## Deployment

### Classification/Regression Workflow

### Clustering Workflow

### Data Product Pipeline